# Find patterns by building
In this notebook the following tasks will be accomplished:

1. Create arrays for heatmap, one per meter-site.
2. Find errors in those arrays.
    - Good fit: `rmsle_scaled` < 0.1
    - In range errors (A): 0.1 <= `rmsle_scaled` <= 0.3
      - single buiding in range long term (A1): time_consequitve_error > 3 days
      - single building in range mid term (A2): 1 day < time_consequitve_error <= 3 days
      - single building in range short term (A3): time_consequitve_error = 1
      - single building in range fluctuatiom (A4):
    - Out of range errors (B): `rmsle_scaled` > 0.3
      - single buiding out of range long term (B1): time_consequitve_error > 3 days
      - single building out of range mid term (B2): 1 day < time_consequitve_error <= 3 days
      - single building out of range short term (B3): time_consequitve_error = 1
      - single building out of range fluctuatiom (B4):

In [1]:
import sys
sys.path.append("..\\source\\")
import utils as utils
import glob

# Data and numbers
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.preprocessing import MinMaxScaler

# Visualization
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import ticker
import matplotlib.dates as mdates
%matplotlib inline

In [2]:
path_data = "..\\data\\processed\\summary\\"
path_leak = "..\\data\\leaked\\"
path_meta = "..\\data\\original\\metadata\\"
path_arrays = "..\\data\\processed\\arrays\\"
path_res = "..\\results\\by_bdg\\"

# Functions

In [3]:
def heatmaps_comp(df,df_bool):

    """This function plots two heatmaps side by side:
        - The original heatmap\n
        - A heatmap where only values over ´tresh_error´ are colored\n

        df: original values for a meter-site.
        df_bool: masked df for that meter-site, where only values over `tresh_error` are 1 (otherwise, 0)\n
    """
    
    fig, axes = plt.subplots(1, 2, sharex = True, sharey=True, figsize=(16,8))
    axes = axes.flatten()

    # Get the data
    y = np.linspace(0, len(df), len(df)+1)
    x = pd.date_range(start='2017-01-01', end='2018-12-31')
    cmap = plt.get_cmap('YlOrRd')

    for i,data in enumerate([df_bool,df]):
        
        # Plot
        ax = axes[i]
        data = data
        qmesh = ax.pcolormesh(x, y, data, cmap=cmap, rasterized=True, vmin=0, vmax=1)

        # Axis
        plt.locator_params(axis='y', nbins=len(list(data.index)) + 1)
        ax.axis('tight') 
        ax.xaxis_date() # Set up as dates
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%b-%y')) # set date's format
        ax.set_yticklabels(list(df_bool.index)) # omit building ID on y axis

    # Color bar  
    cbar = fig.colorbar(qmesh, ax=ax)
    cbar.set_label('Min-Max scaled RMSLE')

    fig.suptitle(f"{meter} - site {site}", y = 1.015, fontsize=16)
    plt.tight_layout()
    plt.subplots_adjust(bottom=0.12)
    #plt.pcolormesh(df, cmap='YlOrRd', rasterized=True)

    return fig

In [4]:
def GoodFit(file_path, t_error, name, plot=False):

    # Load data
    df = pd.read_csv(file).set_index("building_id")
    # Color only error greater than treshold
    df_bool = (df > t_error).astype(int)

    if plot==True:
        fig = heatmaps_comp(df,df_bool)

    df_bool = df_bool.replace([0,1],[1,np.nan])

    return df_bool

In [5]:
def error_1(file, t_error_good, t_error, t_days, name, plot=False):
    """
    Calculates errors type 1 in file: consecutive errors longer than `t_days`.

    file: (string) path to meter-site normalizes RMSLE array (building_id vs. date).\n
    t_error_good: (float) threshold for RMSLE value; values below this are considered a good fit.\n
    t_error: (float) threshold for RMSLE value.\n
    t_day: (int) number of minimum consecutive days to consider it a type 1 error.\n
    name: (string) "A" or "B". If "A", in range values are selected. If "B", out of range values.\n
    plot: (bool) wheter to plot or not.\n

    returns:
    A sparse dataframe, with 1 in those buildings-dates an error ocurs.\n
    """
    # Load file
    df = pd.read_csv(file,index_col="building_id")
    # Keep only selected region
    if "A" in name:
        df_bool = ((df >= t_error_good) & (df <= t_error)).astype(int)
    elif "B" in name:
        df_bool = (df > t_error).astype(int).astype(int)
    else:
        print("Error not recognized.")
        return None

    # Create empty df to assign errors
    df_empty = df[df<0]

    ### GET HIGH ERRORS BY BDG ###
    dfs = []
    df_bdg = df_bool.T

    for col in df_bdg.columns:
        # Select one building
        df1 = df_bdg[[col]]
        # Tag groups of consequtive equal numbers
        df1['grp'] = (df1[col] != df1[col].shift()).cumsum()
        # Filter values == 1 (error higher than t_error)
        df1 = df1.loc[df1[col] == 1,].reset_index().rename(columns={"index":"date"}, index={"building_id":"index"})[["date","grp"]]
        # Add bdg number
        df1["building_id"] = col
        # Get buildings with high error during period longer than t_days
        by_group = df1[["date","grp"]].groupby("grp").count().reset_index() # group
        groups = list(by_group.loc[by_group.date > t_days, "grp"]) # get list of groups
        mt = df1[df1.grp.isin(groups) == True] # filter and get only those groups
        mt["error"] = 1
        # Append to list
        dfs.append(mt)

    # Concat all
    df2 = pd.concat(dfs)
    # Pivot
    df2 = df2.pivot(index="date",columns="building_id", values="error").T

    # Replace on df_empty
    df_empty.update(df2)

    if plot == True:
        fig = heatmaps_comp(df,df_empty.replace(np.nan,0))

    return df_empty

In [6]:
def error_2(file, t_error_good, t_error, t_days, name, plot=False):
    """
    Calculates errors type 2 in file: consecutive errors longer than 1 day and shorter or equal than `t_days`.

    file: (string) path to meter-site normalizes RMSLE array (building_id vs. date).\n
    t_error_good: (float) threshold for RMSLE value; values below this are considered a good fit.\n
    t_error: (float) threshold for RMSLE value.\n
    t_day: (int) number of minimum consecutive days to consider it a type 1 error.\n
    name: (string) "A" or "B". If "A", in range values are selected. If "B", out of range values.\n
    plot: (bool) wheter to plot or not.\n

    returns:
    A sparse dataframe, with 1 in those buildings-dates an error ocurs.\n
    """
    # Load file
    df = pd.read_csv(file,index_col="building_id")
    # Keep only selected region
    if "A" in name:
        df_bool = ((df >= t_error_good) & (df <= t_error)).astype(int)
    elif "B" in name:
        df_bool = (df > t_error).astype(int).astype(int)
    else:
        print("Error not recognized.")
        return None
    

    # Create empty df to assign errors
    df_empty = df[df<0]

    ### GET HIGH ERRORS BY BDG ###
    dfs = []
    df_bdg = df_bool.T

    for col in df_bdg.columns:
        # Select one building
        df1 = df_bdg[[col]]
        # Tag groups of consequtive equal numbers
        df1['grp'] = (df1[col] != df1[col].shift()).cumsum()
        # Filter values == 1 (error higher than t_error)
        df1 = df1.loc[df1[col] == 1,].reset_index().rename(columns={"index":"date"}, index={"building_id":"index"})[["date","grp"]]
        # Add bdg number
        df1["building_id"] = col
        # Get buildings with high error during period longer than t_dates
        by_group = df1[["date","grp"]].groupby("grp").count().reset_index() # group
        groups = list(by_group.loc[(by_group.date > 1) & (by_group.date <= t_days), "grp"]) # get list of groups
        mt = df1[df1.grp.isin(groups) == True] # filter and get only those groups
        mt["error"] = 1
        # Append to list
        dfs.append(mt)

    # Concat all
    df2 = pd.concat(dfs)
    # Pivot
    df2 = df2.pivot(index="date",columns="building_id", values="error").T

    # Replace on df_empty
    df_empty.update(df2)

    if plot == True:
        fig = heatmaps_comp(df,df_empty.replace(np.nan,0))


    return df_empty

In [7]:
def error_3(file, t_error_good, t_error, name, plot=False):
    """
    Calculates errors type 2 in file: errors during 1 day.

    file: (string) path to meter-site normalizes RMSLE array (building_id vs. date).\n
    t_error_good: (float) threshold for RMSLE value; values below this are considered a good fit.\n
    t_error: (float) threshold for RMSLE value.\n
    t_day: (int) number of minimum consecutive days to consider it a type 1 error.\n
    name: (string) "A" or "B". If "A", in range values are selected. If "B", out of range values.\n
    plot: (bool) wheter to plot or not.\n

    returns:
    A sparse dataframe, with 1 in those buildings-dates an error ocurs.\n
    """
    # Load file
    df = pd.read_csv(file,index_col="building_id")
    # Keep only selected region
    if "A" in name:
        df_bool = ((df >= t_error_good) & (df <= t_error)).astype(int)
    else:
        df_bool = (df > t_error).astype(int).astype(int)

    # Create empty df to assign errors
    df_empty = df[df<0]

    ### GET HIGH ERRORS BY BDG ###
    dfs = []
    df_bdg = df_bool.T

    for col in df_bdg.columns:
        # Select one building
        df1 = df_bdg[[col]]
        # Tag groups of consequtive equal numbers
        df1['grp'] = (df1[col] != df1[col].shift()).cumsum()
        # Filter values == 1 (error higher than t_error)
        df1 = df1.loc[df1[col] == 1,].reset_index().rename(columns={"index":"date"}, index={"building_id":"index"})[["date","grp"]]
        # Add bdg number
        df1["building_id"] = col
        # Get buildings with high error during period longer than t_dates
        by_group = df1[["date","grp"]].groupby("grp").count().reset_index() # group
        groups = list(by_group.loc[by_group.date == 1, "grp"]) # get list of groups
        mt = df1[df1.grp.isin(groups) == True] # filter and get only those groups
        mt["error"] = 1
        # Append to list
        dfs.append(mt)

    # Concat all
    df2 = pd.concat(dfs)
    # Pivot
    df2 = df2.pivot(index="date",columns="building_id", values="error").T

    # Replace on df_empty
    df_empty.update(df2)

    if plot == True:
        fig = heatmaps_comp(df,df_empty.replace(np.nan,0))


    return df_empty

In [8]:
def error_4(file, t_error_good, t_error, w, t_w, plot=False):
    """
    Calculates errors type 4 in file: fluctuating errors, shorter than `t_days`.\n

    file: path to meter-site short term error (building_id vs. date).\n
    t_error_good: (float) threshold for RMSLE value; values below this are considered a good fit.\n
    t_error: (float) threshold for RMSLE value.\n
    w: time window to check error.\n
    t_w: proportion of error during time window.\n
    name: (string) "A" or "B". If "A", in range values are selected. If "B", out of range values.\n
    plot: (bool) wheter to plot or not.\n

    returns:
    A sparse dataframe, with 1 in those buildings-dates an error ocurs.\n
    """
    # Load file
    df = pd.read_csv(file,index_col="building_id")
    # Color only error greater than treshold
    df_bool = df.replace(np.nan,0)
    
    """# Keep only selected region
    if "A" in name:
        df_bool = ((df >= t_error_good) & (df <= t_error)).astype(int)
    elif "B" in name:
        df_bool = (df > t_error).astype(int).astype(int)
    else:
        print("Error not recognized.")
        return None"""
    
    ### GET HIGH ERRORS BY BDG ###
    dfs = []
    df_bdg = df_bool.T

    ### CHECK IF t_w PROPORTION IS EXCEED DURIN w TIME PERIOD ###
    for col in df_bdg.columns:
        # Select one building
        df1 = df_bdg[[col]].reset_index().rename(columns={"index":'date'}).fillna(0)
        # Calculate rolling sum
        df1["rol_sum"] = df1[col].rolling(w).sum()
        # Get index of last row of windows, which sum is over threshold
        idx = df1[df1["rol_sum"] > w*t_w].index
        # Mark whole window
        if len(idx) > 0:
            for ix in list(idx):
                i0 = ix-w
                it = ix
                df1.loc[i0:it,"mark"] = df1.loc[i0:it,col] #copy original values in window
        else:
            df1["mark"] = np.nan
        # Rename and complete df
        df1 = df1.set_index("date").drop([col,"rol_sum"],axis=1).rename(columns={"mark":col})
        # Append
        dfs.append(df1)

    #Create errors df
    df_error = pd.concat(dfs,axis=1).replace(0,np.nan).T

    if plot == True:
        fig = heatmaps_comp(df,df_error.replace(np.nan,0))

    return df_error

# Create arrays

In [7]:
"""meters = ["electricity","chilledwater","hotwater","steam"]
group = "site_id"
metric = "RMSLE"

for meter in meters:

    # Load data
    df = pd.read_pickle(path_data + f"{meter}_RMSLE.pickle.gz")

    # Remove leaked buidings
    leak = pd.read_csv(path_leak + f"leak_{meter}.csv")
    df = df[df.building_id.isin(leak.building_id) == False]

    # Datetime object
    df.timestamp = pd.to_datetime(df.timestamp, format="%Y-%m-%d")

    # Complete missing dates
    df = utils.complete_data(df,"2017-01-01","2018-12-31")

    # Merge with metadata
    meta_bdg = pd.read_csv(path_meta + "building_metadata.csv")
    df = pd.merge(df, meta_bdg, how="left", on="building_id")

    # Scale metric between 0 and 1
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_rmsle = scaler.fit_transform(np.array(df.rmsle).reshape(-1, 1))
    # Add to df
    df["rmsle_scaled"] = scaled_rmsle
    # Print limits
    print(f"Meter: {meter}")
    print(f"Min rmsle: {df.rmsle.min()}. Max rmsle: {df.rmsle.max()}")
    print(f"Min rmsle_scaled: {df.rmsle_scaled.min()}. Max rmsle_scaled: {df.rmsle_scaled.max()}")
    print("")

    # Create arrays
    group_name = list(df[group].unique())

    for i,j in enumerate(group_name):

        # Filter data
        df_grouped = df[df[group] == j]

        # Pivot data
        pivot_df = df_grouped.pivot(columns="timestamp", index="building_id", values="rmsle_scaled")

        # Sort in descending order (sum RMSLE)
        pivot_df["sum"] = np.sum(pivot_df,axis=1).tolist()
        pivot_df = pivot_df.sort_values("sum")
        pivot_df.drop("sum",axis=1, inplace=True)

        # Save df
        pivot_df.to_csv(path_arrays + f"{meter}_{metric}_site_{j}.csv")"""

Meter: electricity
Min rmsle: 0.03999936208128929. Max rmsle: 8.138672828674316
Min rmsle_scaled: 0.0. Max rmsle_scaled: 1.0

Meter: chilledwater
Min rmsle: 0.07809336483478546. Max rmsle: 8.397016525268555
Min rmsle_scaled: 0.0. Max rmsle_scaled: 0.9999999999999999

Meter: hotwater
Min rmsle: 0.11213328689336777. Max rmsle: 8.06053352355957
Min rmsle_scaled: 0.0. Max rmsle_scaled: 1.0

Meter: steam
Min rmsle: 0.0737953633069992. Max rmsle: 9.318635940551758
Min rmsle_scaled: 0.0. Max rmsle_scaled: 1.0



# Choose meter

In [9]:
meters = ["chilledwater","electricity","hotwater","steam"]

# Good fit (Z)

In [29]:
for meter in meters:
    #Get list of files
    files = glob.glob(path_arrays + f"{meter}*.csv")

    for file in files:
        # Site id
        site = file.split("\\")[-1].split("_")[-1].split(".")[0]
        print(f"{meter} - Site {site}")
        #Create df
        df = GoodFit(file, 0.1, "Z", plot=False)
        # Save df
        df.to_csv(f"{path_res}\\{meter}_Z_site{site}.csv")

    print()

chilledwater - Site 10
chilledwater - Site 11
chilledwater - Site 13
chilledwater - Site 14
chilledwater - Site 6
chilledwater - Site 7
chilledwater - Site 9

electricity - Site 10
electricity - Site 11
electricity - Site 12
electricity - Site 13
electricity - Site 14
electricity - Site 3
electricity - Site 4
electricity - Site 5
electricity - Site 6
electricity - Site 7
electricity - Site 8
electricity - Site 9

hotwater - Site 1
hotwater - Site 10
hotwater - Site 11
hotwater - Site 14
hotwater - Site 7

steam - Site 13
steam - Site 14
steam - Site 6
steam - Site 7
steam - Site 9



# In range errors (A)

## A1

In [54]:
for meter in meters:
    #Get list of files
    files = glob.glob(path_arrays + f"{meter}*.csv")

    for file in files:
        # Site id
        site = file.split("\\")[-1].split("_")[-1].split(".")[0]
        print(f"{meter} - site {site}")
        # Create df
        df = error_1(file,0.1, 0.3, 3,"A1", plot=False)
        # Save df
        df.to_csv(f"{path_res}\\{meter}_A1_site{site}.csv")

    print()

chilledwater - site 10
chilledwater - site 11
chilledwater - site 13
chilledwater - site 14
chilledwater - site 6
chilledwater - site 7
chilledwater - site 9

electricity - site 10
electricity - site 11
electricity - site 12
electricity - site 13
electricity - site 14
electricity - site 3
electricity - site 4
electricity - site 5
electricity - site 6
electricity - site 7
electricity - site 8
electricity - site 9

hotwater - site 1
hotwater - site 10
hotwater - site 11
hotwater - site 14
hotwater - site 7

steam - site 13
steam - site 14
steam - site 6
steam - site 7
steam - site 9



## A2

In [44]:
for meter in meters:
    #Get list of files
    files = glob.glob(path_arrays + f"{meter}*.csv")

    for file in files:
        # Site id
        site = file.split("\\")[-1].split("_")[-1].split(".")[0]
        print(f"{meter} - site {site}")
        # Create df
        df = error_2(file,0.1, 0.3, 3, "A2", plot=False)
        # Save df
        df.to_csv(f"{path_res}\\{meter}_A2_site{site}.csv")

    print()

chilledwater - site 10
chilledwater - site 11
chilledwater - site 13
chilledwater - site 14
chilledwater - site 6
chilledwater - site 7
chilledwater - site 9

electricity - site 10
electricity - site 11
electricity - site 12
electricity - site 13
electricity - site 14
electricity - site 3
electricity - site 4
electricity - site 5
electricity - site 6
electricity - site 7
electricity - site 8
electricity - site 9

hotwater - site 1
hotwater - site 10
hotwater - site 11
hotwater - site 14
hotwater - site 7

steam - site 13
steam - site 14
steam - site 6
steam - site 7
steam - site 9



## A3

In [48]:
for meter in meters:
    #Get list of files
    files = glob.glob(path_arrays + f"{meter}*.csv")

    for file in files:
        # Site id
        site = file.split("\\")[-1].split("_")[-1].split(".")[0]
        print(f"{meter} - site {site}")
        # Create df
        df = error_3(file,0.1, 0.3,"A3", plot=False)
        # Save df
        df.to_csv(f"{path_res}\\{meter}_A3_site{site}.csv")

    print()

chilledwater - site 10
chilledwater - site 11
chilledwater - site 13
chilledwater - site 14
chilledwater - site 6
chilledwater - site 7
chilledwater - site 9

electricity - site 10
electricity - site 11
electricity - site 12
electricity - site 13
electricity - site 14
electricity - site 3
electricity - site 4
electricity - site 5
electricity - site 6
electricity - site 7
electricity - site 8
electricity - site 9

hotwater - site 1
hotwater - site 10
hotwater - site 11
hotwater - site 14
hotwater - site 7

steam - site 13
steam - site 14
steam - site 6
steam - site 7
steam - site 9



## A4
This error is based on A3 errors.

In [12]:
for meter in meters:
    #Get list of files
    files = glob.glob(path_res + f"{meter}_A3_*.csv")

    for file in files:
        # Site id
        site = file.split("\\")[-1].split("_")[-1].split(".")[0].split("site")[1]
        print(f"{meter} - site {site}")
        # Create df
        df = error_4(file,0.1, 0.3, 30, 0.1, plot=False)
        # Save df
        df.to_csv(f"{path_res}\\{meter}_A4_site{site}.csv")

    print()

chilledwater - site 10
chilledwater - site 11
chilledwater - site 13
chilledwater - site 14
chilledwater - site 6
chilledwater - site 7
chilledwater - site 9

electricity - site 10
electricity - site 11
electricity - site 12
electricity - site 13
electricity - site 14
electricity - site 3
electricity - site 4
electricity - site 5
electricity - site 6
electricity - site 7
electricity - site 8
electricity - site 9

hotwater - site 1
hotwater - site 10
hotwater - site 11
hotwater - site 14
hotwater - site 7

steam - site 13
steam - site 14
steam - site 6
steam - site 7
steam - site 9



# Out of range errors (B)

## B1

In [51]:
for meter in meters:
    #Get list of files
    files = glob.glob(path_arrays + f"{meter}*.csv")

    for file in files:
        # Site id
        site = file.split("\\")[-1].split("_")[-1].split(".")[0]
        print(f"{meter} - site {site}")
        # Create df
        df = error_1(file,0.1, 0.3, 3,"B1", plot=False)
        # Save df
        df.to_csv(f"{path_res}\\{meter}_B1_site{site}.csv")

    print()

chilledwater - site 10
chilledwater - site 11
chilledwater - site 13
chilledwater - site 14
chilledwater - site 6
chilledwater - site 7
chilledwater - site 9

electricity - site 10
electricity - site 11
electricity - site 12
electricity - site 13
electricity - site 14
electricity - site 3
electricity - site 4
electricity - site 5
electricity - site 6
electricity - site 7
electricity - site 8
electricity - site 9

hotwater - site 1
hotwater - site 10
hotwater - site 11
hotwater - site 14
hotwater - site 7

steam - site 13
steam - site 14
steam - site 6
steam - site 7
steam - site 9



## B2

In [52]:
for meter in meters:
    #Get list of files
    files = glob.glob(path_arrays + f"{meter}*.csv")

    for file in files:
        # Site id
        site = file.split("\\")[-1].split("_")[-1].split(".")[0]
        print(f"{meter} - site {site}")
        # Create df
        df = error_2(file,0.1, 0.3, 3, "B2", plot=False)
        # Save df
        df.to_csv(f"{path_res}\\{meter}_B2_site{site}.csv")

    print()

chilledwater - site 10
chilledwater - site 11
chilledwater - site 13
chilledwater - site 14
chilledwater - site 6
chilledwater - site 7
chilledwater - site 9

electricity - site 10
electricity - site 11
electricity - site 12
electricity - site 13
electricity - site 14
electricity - site 3
electricity - site 4
electricity - site 5
electricity - site 6
electricity - site 7
electricity - site 8
electricity - site 9

hotwater - site 1
hotwater - site 10
hotwater - site 11
hotwater - site 14
hotwater - site 7

steam - site 13
steam - site 14
steam - site 6
steam - site 7
steam - site 9



## B3

In [53]:
for meter in meters:
    #Get list of files
    files = glob.glob(path_arrays + f"{meter}*.csv")

    for file in files:
        # Site id
        site = file.split("\\")[-1].split("_")[-1].split(".")[0]
        print(f"{meter} - site {site}")
        # Create df
        df = error_3(file,0.1, 0.3,"B3", plot=False)
        # Save df
        df.to_csv(f"{path_res}\\{meter}_B3_site{site}.csv")

    print()

chilledwater - site 10
chilledwater - site 11
chilledwater - site 13
chilledwater - site 14
chilledwater - site 6
chilledwater - site 7
chilledwater - site 9

electricity - site 10
electricity - site 11
electricity - site 12
electricity - site 13
electricity - site 14
electricity - site 3
electricity - site 4
electricity - site 5
electricity - site 6
electricity - site 7
electricity - site 8
electricity - site 9

hotwater - site 1
hotwater - site 10
hotwater - site 11
hotwater - site 14
hotwater - site 7

steam - site 13
steam - site 14
steam - site 6
steam - site 7
steam - site 9



## B4

In [18]:
for meter in meters:
    #Get list of files
    files = glob.glob(path_res + f"{meter}_B3_*.csv")

    for file in files:
        # Site id
        site = file.split("\\")[-1].split("_")[-1].split(".")[0].split("site")[1]
        print(f"{meter} - site {site}")
        # Create df
        df = error_4(file,0.1, 0.3, 60, 0.1, plot=False)
        # Save df
        df.to_csv(f"{path_res}\\{meter}_B4_site{site}.csv")

    print()

chilledwater - site 10
chilledwater - site 11
chilledwater - site 13
chilledwater - site 14
chilledwater - site 6
chilledwater - site 7
chilledwater - site 9

electricity - site 10
electricity - site 11
electricity - site 12
electricity - site 13
electricity - site 14
electricity - site 3
electricity - site 4
electricity - site 5
electricity - site 6
electricity - site 7
electricity - site 8
electricity - site 9

hotwater - site 1
hotwater - site 10
hotwater - site 11
hotwater - site 14
hotwater - site 7

steam - site 13
steam - site 14
steam - site 6
steam - site 7
steam - site 9

